In [61]:
from pymongo import MongoClient
import pprint as pp
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import os

In [46]:
def convert_to_date(year, day_of_the_year):
    '''
    parameters:
    -----------
    year: Integer
    day_of_the_year: an integer ranging from 1 to 366
    
    returns:
    --------
    a python date object the indicates the date
    '''
    return dt.datetime(year, 1, 1) + dt.timedelta(day_of_the_year - 1)

In [47]:
client = MongoClient("mongodb://localhost:27017/")
db = client.github_watch
gt_watch = db.gt_watch_no_fork

lowDate = dt.datetime(2012,4,1)
highDate = dt.datetime(2014,12,31)

In [48]:
doc = gt_watch.find_one()
pp.pprint(doc)

{u'_id': ObjectId('5856a9350355e82f7dbcd676'),
 u'created_at': datetime.datetime(2012, 4, 1, 19, 56, 59),
 u'repository_name': u'X',
 u'repository_watchers': 39.0}


In [49]:
pipeline_watch = [{"$match":{"created_at":{"$gt":lowDate,"$lt":highDate},"repository_name":{"$exists":True}}},\
            {"$group":{"_id":{"project":"$repository_name","year":{"$year":"$created_at"},"month":{"$month":"$created_at"},"date":{"$dayOfYear":"$created_at"}},"cnt":{"$sum":1}}},\
            {"$sort":{"cnt":-1}},\
            {"$sort":{"_id.date":1}},\
            {"$sort":{"_id.year":1}}]#,\
            # {"$limit":5000}]

In [50]:
results = gt_watch.aggregate(pipeline_watch, allowDiskUse=True)

In [51]:
runningWeek = 0
last_day = dt.datetime(1998,1,1)
all_date_list = list()
rankList = list()
currentRankList = list()

In [ ]:
for result in results:
    ## Create a 2d list, each row contains project in descending order
    ## of their rank based on how many emails they sent or received
    day_current_document = result["_id"]["date"]
    year_current_document = result["_id"]["year"]
    current_last_day = convert_to_date(year_current_document, day_current_document)
    
    if current_last_day > last_day:
        runningWeek += 1
        print "running day ", runningWeek
        rankList.append(currentRankList)
        last_day = current_last_day
        currentRankList = list()
        currentRankList.append([result["_id"]['project'], str(result["cnt"])])
        all_date_list.append(current_last_day)
    else:
        currentRankList.append([result["_id"]['project'], str(result["cnt"])])
    #pp.pprint(result)


In [58]:
len(rankList)

1003

In [59]:
len(all_date_list)

1003

In [56]:
rankList.pop(0)

[]

In [57]:
all_date_list.pop()

datetime.datetime(2014, 12, 30, 0, 0)

In [62]:
directory = "data_rank_github_watch_no_fork_daily"
if not os.path.exists(directory):
    os.makedirs(directory)

In [63]:
filename_all_dates = 'time_list_daily_github_watch_no_fork.txt'
with open(filename_all_dates,'w') as f_date:
    for i,date in enumerate(all_date_list):
        current_date_string = date.strftime('%d.%m.%Y')
        f_date.write(current_date_string + "\n")
        filename_rank_current_week = directory + "/" + current_date_string
        with open(filename_rank_current_week, 'w') as f_weekly_rank:
            for project_and_count in rankList[i]:
                f_weekly_rank.write("\t".join(project_and_count)+"\n")